In [ ]:
import xarray as xr
from pystac import Item
from odc.stac import load

from pathlib import Path

from datetime import datetime

import sys
import os
sys.path.insert(0, "..")

from ghrsst.cogger import load_data

In [ ]:
LOCAL_FOLDER = "data"
LOCAL_FILE_DATE = datetime(2023, 11, 6)
LOCAL_FILE = str(Path(os.path.abspath("")).parent / LOCAL_FOLDER)

data = load_data(LOCAL_FILE_DATE, LOCAL_FILE)

data

In [ ]:
for x in data.odc.geobox.affine:
    print(float(x))

In [ ]:
from odc.geo.xr import xr_coords
from affine import Affine
from odc.geo.geobox import GeoBox

new_affine = Affine(0.01, 0.0, -180.0, 0.0, 0.01, -89.995, 0.0, 0.0, 1.0)
new_geobox = GeoBox(data.odc.geobox.shape, new_affine, data.odc.geobox.crs)

new_coords = xr_coords(new_geobox)
# new_coords 

# Rename lat to latitude and lon to longitude
new_data = data.rename({"lat": "latitude", "lon": "longitude"}).assign_coords(new_coords)
[float(x) for x in new_data.odc.geobox.affine]

In [ ]:
float(data["lon"].values[0]), float(data["lon"].values[-1])

In [ ]:
new_array = range(-179995, 180000, 10)
array_floats = [float(x) / 1000 for x in new_array]

array_floats[0], array_floats[-1]

data["lon"] = array_floats
data = data.odc.assign_crs("EPSG:4326")
list(data.odc.geobox.affine)

In [ ]:
lat_1000s = range(-89990, 90000, 10)
lats = [float(x) / 1000.0 for x in lat_1000s]
data["lat"] = lats

data = data.odc.assign_crs("EPSG:4326")

print(lats[-1])

print(list(data.odc.affine))
print(f"{data.lat.values[0]:0.5f}), {data.lat.values[-1]:0.5f}")

In [ ]:
float(data.odc.geobox.resolution.x)

In [ ]:
from odc.geo.geobox import GeoBox
from odc.geo.xr import xr_reproject
from affine import Affine


new_affine = Affine(0.01, 0.0, -180.0, 0.0, 0.01, -89.995, 0.0, 0.0, 1.0)
new_geobox = GeoBox(data.odc.geobox.shape, new_affine, data.odc.geobox.crs)

reprojected = xr_reproject(data, new_geobox, resampling="nearest")

float(reprojected["longitude"][0].values), float(reprojected["longitude"][-1].values)

In [ ]:
diff = (data.analysed_sst.values - reprojected.analysed_sst.values)

In [ ]:
from xarray import DataArray

data["diff"] = DataArray(diff, dims=["time", "lat", "lon"])

data["diff"].squeeze().plot.imshow()

In [ ]:
data.odc.geobox.affine = new_affine

In [ ]:
float(reprojected["longitude"][0].values), float(reprojected["longitude"][-1].values)

In [ ]:
item = Item.from_file("data/output/20231106090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.stac-item.json")
item

In [ ]:
data = load([item], chunks={})
data

In [ ]:
data.analysed_sst.isel(time=0).plot.imshow(size=10, cmap='jet')